# MLP MNIST
Hier werden das erste Mal im MNIST nun Algotihmen aus dem 
Multilayer Perceptron (MLP) for multi-class softmax classification:

In [ ]:
# Import Libraries
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import model_from_json
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Hyperparamter Upfront 
batch_size = 128 # Bei MNIST wähle ich die 128 immer ident
num_classes = 10 # Vorgabe aus Datensatz
epochs = 20 # Wird in allen Modellen geählt

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
# Für MLP brauch ich einen Eingangsvektor => Reshaping

In [ ]:
x_train = x_train.reshape(60000, 784) # MLP benötigt InputVektor und keine Matrix => reshape
x_test = x_test.reshape(10000, 784) # MLP benötigt InputVektor und keine Matrix => reshape
x_train = x_train.astype('float32') # Standard 
x_test = x_test.astype('float32') # Standard 
x_train /= 255 # skalieren
x_test /= 255 # Skalieren

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#Datenvektorisierung
y_train = keras.utils.to_categorical(y_train, num_classes) # Standard bei NN
y_test = keras.utils.to_categorical(y_test, num_classes)# Standard bei NN

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Prüfen ob bereits ein Model bereits trainiert und gespeichert ist, im Falle, dass noch kein Modell vorliegt neu erstellen
modelMLP = Path("./MODEL/modelMLP.json")
if modelMLP.is_file():
   # load json and create model
    json_file = open('./MODEL/modelMLP.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    modelMLP = model_from_json(loaded_model_json)
    # load weights into new model
    modelMLP.load_weights("./MODEL/modelMLP.h5")
    print("Loaded model from disk")
else:
    modelMLP = Sequential()
    modelMLP.add(Dense(512, activation='relu', input_shape=(784,)))
    modelMLP.add(Dropout(0.2))
    modelMLP.add(Dense(512, activation='relu'))
    modelMLP.add(Dropout(0.2))
    modelMLP.add(Dense(num_classes, activation='softmax')) # hier der letzte Layer hat die Anzahl der möglichen klassen & die Softmax Aktivierung
    modelMLP.summary()

In [ ]:
#
modelMLP.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


In [ ]:
# Ausführen des Modelles
history = modelMLP.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, # show training progress during training Possible = 0,1,2
                    validation_data=(x_test, y_test))

In [ ]:
# Save trained Model to disk via JSON
model_json = modelMLP.to_json()
with open("./MODEL/modelMLP.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
modelMLP.save_weights("./MODEL/modelMLP.h5")
print("Saved modelMLP to disk")

In [ ]:
score = modelMLP.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Plot training & validation accuracy values

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:

tf.keras.utils.plot_model(
    modelMLP,
    to_file='model.png',
    show_shapes=False,
    show_layer_names=True,
    rankdir='TB'
)